# TensorFlow: Sentiment classification using pre-trained word embeddings

In [ ]:
import os

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tf_keras as keras

print("TF Version: ", tf.__version__)
print("TF Eager mode: ", tf.executing_eagerly())
print("TF GPU is", "available" if tf.config.list_physical_devices("GPU") else "not available")

## Prepare Datasets

In [ ]:
train_ds, val_ds, test_ds = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [ ]:
train_ds = (train_ds
    .shuffle(10000)
    .batch(512)
    .prefetch(tf.data.experimental.AUTOTUNE))
val_ds = (val_ds
    .shuffle(10000)
    .batch(512)
    .prefetch(tf.data.experimental.AUTOTUNE))
test_ds = (test_ds
    .shuffle(10000)
    .batch(512)
    .prefetch(tf.data.experimental.AUTOTUNE))

## Build Model

### Create Model

In [ ]:
# Use pre-training embeddings vectors:
#   - google/nnlm-en-dim50/2
#   - google/nnlm-en-dim128/2
#   - google/nnlm-en-dim128-with-normalization/2
#   - google/universal-sentence-encoder/4

embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"

# Create a layer with pre-trained embeddings
layer = hub.KerasLayer(
    embedding,
    input_shape=[],
    dtype=tf.string,
    trainable=True)

In [ ]:
# Get word embeddings for three samples
train_x, _ = next(iter(train_ds))
embeddings = layer(train_x[:3])
print(f"Shape: {embeddings.shape}")

In [ ]:
# Create a model using correct keras package
model = keras.Sequential([
    layer,
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

In [ ]:
model.summary()

In [ ]:
# Compile model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Fit Model

In [ ]:
%%time
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    verbose=1)

### Evaluate Model

In [ ]:
results = model.evaluate(test_ds, verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))